# archive to folder


## tests
ARCHIVE_URL = "https://s3.wasabisys.com/iguazio/data/cats-vs-dogs/cats-vs-dogs-labeling-demo.zip"

fn = import_function("hub://open_archive")
fn.run(inputs={'archive_url': ARCHIVE_URL})
